# Tool Calling Dependency Injection

[Dependency Injection](https://en.wikipedia.org/wiki/Dependency_injection) is a secure way to connect external functions to agents without exposing sensitive data such as passwords, tokens, or personal information. This approach ensures that sensitive information remains protected while still allowing agents to perform their tasks effectively, even when working with large language models (LLMs).

In this guide, we’ll explore how to build secure workflows that handle sensitive data safely.

As an example, we’ll create an agent that retrieves user's account balance. The best part is that sensitive data like username and password are never shared with the LLM. Instead, it’s securely injected directly into the function at runtime, keeping it safe while maintaining seamless functionality.

## Why Use Dependency Injection?

When working with LLMs, **security is paramount**. There are several types of sensitive information that we want to keep out of the LLM’s reach:

- **Passwords or tokens**: These could be exposed through [prompt injection attacks](https://en.wikipedia.org/wiki/Prompt_injection).
- **Personal information**: Access to this data might fall under strict regulations, such as the [EU AI Act](https://www.europarl.europa.eu/topics/en/article/20230601STO93804/eu-ai-act-first-regulation-on-artificial-intelligence).

Dependency injection offers a robust solution by isolating sensitive data while enabling your agents to function effectively.

## Why Dependency Injection Is Essential

Here’s why dependency injection is a game-changer for secure LLM workflows:

- **Enhanced Security**: Your sensitive data is never directly exposed to the LLM.
- **Simplified Development**: Secure data can be seamlessly accessed by functions without requiring complex configurations.
- **Unmatched Flexibility**: It supports safe integration of diverse workflows, allowing you to scale and adapt with ease.

In this guide, we’ll explore how to set up dependency injection and build secure workflows. Let’s dive in!

## Installation

To install `AG2`, simply run the following command:

```bash
pip install ag2
```


## Imports

The functionality demonstrated in this guide is located in the `autogen.tools.dependency_injection` module. This module provides key components for dependency injection:

- `BaseContext`: abstract base class used to define and encapsulate data contexts, such as user account information, which can then be injected into functions or agents securely.
- `Depends`: a function used to declare and inject dependencies, either from a context (like `BaseContext`) or a function, ensuring sensitive data is provided securely without direct exposure.

In [1]:
import os
from typing import Annotated, Literal

from pydantic import BaseModel

from autogen import GroupChat, GroupChatManager
from autogen.agentchat import ConversableAgent, UserProxyAgent
from autogen.tools.dependency_injection import BaseContext, Depends

## Context and Dependency Setup

### Define a BaseContext Class
We start by defining a `BaseContext` class for accounts. This will act as the base structure for dependency injection.

In [2]:
class Account(BaseContext, BaseModel):
    username: str
    password: str
    currency: Literal["USD", "EUR"] = "USD"


alice_account = Account(username="alice", password="password123")
bob_account = Account(username="bob", password="password456")

account_ballace_dict = {
    (alice_account.username, alice_account.password): 300,
    (bob_account.username, bob_account.password): 200,
}


### Helper Functions
To ensure that the provided account is valid and retrieve its balance, we create two helper functions.

In [3]:
def _verify_account(account: Account):
    if (account.username, account.password) not in account_ballace_dict:
        raise ValueError("Invalid username or password")


def _get_balance(account: Account):
    _verify_account(account)
    return f"Your balance is {account_ballace_dict[(account.username, account.password)]}{account.currency}"

## Injecting BaseContext Parameter

Dependency injection simplifies passing data to a function. Here, we'll inject an `Account` instance into a function automatically.


### Agent Configuration

Configure the agents for the interaction.

- `config_list` defines the LLM configurations, including the model and API key.
- `UserProxyAgent` simulates user inputs without requiring actual human interaction (set to `NEVER`).
- `AssistantAgent` represents the AI agent, configured with the LLM settings.

In [4]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
assistant = ConversableAgent(
    name="assistant",
    llm_config={"config_list": config_list},
)
user_proxy = UserProxyAgent(
    name="user_proxy_1",
    human_input_mode="NEVER",
    llm_config=False,
)


### Register the Function with Dependency Injection
We register a function where the account information for `bob` is injected as a dependency.

**Note:** You can also use `account: Account = Depends(bob_account)` as an alternative syntax.

In [5]:
@user_proxy.register_for_execution()
@assistant.register_for_llm(description="Get the balance of the account")
def get_balance_1(
    # Account which will be injected to the function
    account: Annotated[Account, Depends(bob_account)],
    # It is also possible to use the following syntax to define the dependency
    # account: Account = Depends(bob_account),
) -> str:
    return _get_balance(account)

### Initiate the Chat
Finally, we initiate a chat to retrieve the balance.

In [ ]:
user_proxy.initiate_chat(assistant, message="Get users balance", max_turns=2)

## Injecting Parameters Without BaseContext

Sometimes, you might not want to use `BaseContext`. Here's how to inject simple parameters directly.

### Agent Configuration

Configure the agents for the interaction.

- `config_list` defines the LLM configurations, including the model and API key.
- `UserProxyAgent` simulates user inputs without requiring actual human interaction (set to `NEVER`).
- `AssistantAgent` represents the AI agent, configured with the LLM settings.

In [7]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
assistant = ConversableAgent(
    name="assistant",
    llm_config={"config_list": config_list},
)
user_proxy = UserProxyAgent(
    name="user_proxy_1",
    human_input_mode="NEVER",
    llm_config=False,
)

### Register the Function with Direct Parameter Injection
In this section, we demonstrate how to register a function with **direct parameter injection** using `Depends`. Instead of injecting a full context like `Account`, we inject individual parameters (e.g., username and password) directly into the function.

In [8]:
def get_username() -> str:
    return "bob"

def get_password() -> str:
    return "password456"

@user_proxy.register_for_execution()
@assistant.register_for_llm(description="Get the balance of the account")
def get_balance_2(
    username: Annotated[str, Depends(get_username)],
    password: Annotated[str, Depends(get_password)],
) -> str:
    account = Account(username=username, password=password)
    return _get_balance(account)

### Initiate the Chat
As before, initiate a chat to test the function.

In [ ]:
user_proxy.initiate_chat(assistant, message="Get users balance", max_turns=2)

## Using Different Contexts for Different Agents

In this section, we demonstrate how to assign different contexts (e.g., different account data) to different agents in the same group chat. By doing this, we ensure that each assistant interacts with its own unique set of data, such as using `alice_account` for one assistant and `bob_account` for another.

### GroupChat Configuration
Let's configure a `GroupChat` with two assistant agents.


In [10]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
llm_config = {"config_list": config_list}
assistant_1 = ConversableAgent(
    name="assistant_1",
    llm_config={"config_list": config_list},
)
assistant_2 = ConversableAgent(
    name="assistant_2",
    llm_config={"config_list": config_list},
)
user_proxy = UserProxyAgent(
    name="user_proxy_1",
    human_input_mode="NEVER",
    llm_config=False,
)

groupchat = GroupChat(agents=[user_proxy, assistant_1, assistant_2], messages=[], max_round=5)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

### Register Functions for Each Assistant

- For `assistant_1`, we inject the `alice_account` context using `Depends(alice_account)`, ensuring that it retrieves the balance for Alice’s account.
- For `assistant_2`, we inject the `bob_account` context using `Depends(bob_account)`, ensuring that it retrieves the balance for Bob’s account.

In [11]:
@user_proxy.register_for_execution()
@assistant_1.register_for_llm(description="Get the balance of the account")
def get_balance_for_assistant_1(
    account: Annotated[Account, Depends(alice_account)],
) -> str:
    return _get_balance(account)


@user_proxy.register_for_execution()
@assistant_2.register_for_llm(description="Get the balance of the account")
def get_balance_for_assistant_2(
    account: Annotated[Account, Depends(bob_account)],
) -> str:
    return _get_balance(account)

### Initiate the Chat
Finally, initiate the group chat where both assistants respond by using their respective contexts. Each assistant will handle its own task — one will retrieve Alice’s balance, and the other will retrieve Bob’s balance

In [ ]:
message = "Both assistants, please get the balance of the account"
user_proxy.initiate_chat(manager, message=message, max_turns=1)